In [ ]:
import pandas as pd

In [ ]:
df11 = pd.read_excel('/content/q4_metals.xlsx')

In [ ]:
df11.columns

Index(['Business Name', 'Address', 'Question', 'Expected Response', 'Snippets',
       'Url', 'Machine result', 'Machine Snippet', 'Machine url', 'Validation',
       'Metrics', 'UW Review', 'UW Response', 'UW Comments', 'Unnamed: 14'],
      dtype='object')

In [ ]:
scrape=pd.read_excel('/content/question-to-label_output.xlsx')

In [ ]:
scrape.columns=['Unnamed: 0', 'id', 'Idenifier', 'business_name', 'business_dba',
       'business_address', 'business_city', 'business_state',
       'business_zipcode', 'Url', 'webtext']

In [ ]:
scrape['Url']=scrape['Url'].apply(lambda x:eval(x)[0])

In [ ]:
df11=df11.merge(scrape[['Url','webtext']],on='Url',how='left')

In [ ]:
df11

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
!pip install sentence_transformers

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-mpnet-base-v2')


In [ ]:
#Question='Does the insured/applicant process, transmit, collect, record, store or otherwise handle non-employee Protected Health Information (PHI) or other health/fitness information?'
#Question='geomapping, global positioning systems, aerial photography, aerial drones, land surveying,geospacial technologies, GIS,GPS'
Question=' exotic metals, materials, nanoparticles (particles less than 100 nanometers) radioactive materials or materials in a sub-atomic form; or b) operate a Biosafety Level 3 or 4 laboratory'
Q_embeddings = model.encode(Question)
Q_embeddings.shape

(768,)

In [ ]:
df11=df11[~df11['webtext'].isna()]
df11.index=range(len(df11))

In [ ]:
df11['webtext'][0]

In [ ]:
df11

In [ ]:
import sentence_transformers 
from sklearn.metrics.pairwise import cosine_similarity
from nltk import tokenize
a3=[]
a4=[]
for i in range(0,len(df11)):
    print(i)
    A=df11['webtext'][i]
    #B=tokenize.sent_tokenize(A)
    B=A.split('\n')
    df=pd.DataFrame(B)
    df=df.rename(columns = {0: 'Sentence'})
    a1=[]
    a2=[]
    a5=[]
    for j in range(0,len(B)):
        paragraph =B[j]
        res = len(B[j].split())
        if res>3:
          sen_embeddings = model.encode(paragraph)

          #let's calculate cosine similarity for sentence 0:
          sim=cosine_similarity(
              [Q_embeddings],
              sen_embeddings.reshape(1,-1)
          )
          ab=sentence_transformers.util.dot_score(Q_embeddings, sen_embeddings)
          res = len(B[j].split())
          a1.append(sim)
          a2.append(ab)
          a5.append(res)
        #print(sim)

    df['Similarity cos_sim']=pd.Series(a1)  
    df['Similarity Dot']=pd.Series(a2)  
    df['Sen Length']=pd.Series(a5) 
    #df=df[df['Sen Length'] > 6]
    df=df.reset_index()
    if len(df)>1:
      df['Similarity cos_sim']=df['Similarity cos_sim'].str.get(0)
      df['Similarity cos_sim']=df['Similarity cos_sim'].str.get(0)
      df1=df.sort_values(by=['Similarity cos_sim'], ascending=False)
      df2=df1.head(5)
      out = ' '.join(df2["Sentence"])    
      out2=df2["Similarity cos_sim"].mean()
      a3.append(out)
      a4.append(out2)
    else:
      a3.append('')
      a4.append(0)
 

In [ ]:
df11['New Context']=a3   
df11['Avg Sim']=a4  

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
df11.index=range(len(df11))

In [ ]:
df11=df11.sort_values('Avg Sim',ascending=False)

In [ ]:
df11.to_excel('q4_semantic_n_split.xlsx')